In [1]:
import scipy.io
import scipy.sparse.linalg as sp_linalg
import pandas as pd
import networkx as nx
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
name = 'mutag'
node_labels = True
l = 3
num_eig = 3

In [3]:
dataset = scipy.io.loadmat('datasets/'+name+'.mat')
names = list(dataset.keys())[3:]
names


['lmutag', 'mutag']

In [4]:
Graphs = dataset[names[1]][0]
labels = Graphs[0][1][0][0][0].flatten()
total = len(Graphs[0])
labels

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 6., 7., 7.])

In [5]:
def process_labels(lab):
    unique = np.unique(lab.flatten())
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(unique.reshape(-1, 1))
    oneHot = enc.transform(lab.reshape(-1, 1)).toarray()
    return unique, oneHot
process_labels(labels)

(array([3., 6., 7.]),
 array([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.]]))

In [6]:
G = nx.from_numpy_matrix(Graphs[0][0])
#A es la matriz de adjacencia del grafo
A = nx.adjacency_matrix(G).todense()
#d es el arreglo de grados del grafo
d = A * np.ones((G.number_of_nodes(), 1))
#D es la matriz de grados diagonales
print(d.transpose())
D = np.diag(d.transpose().tolist()[0])
D

[[2. 2. 3. 2. 2. 3. 3. 2. 2. 3. 3. 3. 3. 2. 3. 3. 2. 2. 2. 2. 3. 1. 1.]]


/tmp/ipykernel_23017/4021707887.py:3: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G).todense()


array([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0.,

In [7]:
#M es la matriz de markov del grafo
M = np.linalg.inv(D) * A

In [8]:
#np.transpose(d)
# m es la cantidad de arcos del grafo
m = len(G.edges)
#pi es el vector de distribución estacionaria
pi = d.transpose() / (2*m)
#PI es la matriz diagonal de distribución estacionaria
PI = np.diag(pi.tolist()[0])

In [9]:
def ro(t, P, p, Me):
    return P * np.linalg.matrix_power(Me, t) - pi.transpose() * pi
print(PI.shape, pi.shape, M.shape)
ro(1, PI, pi, M)

(23, 23) (1, 23) (23, 23)


matrix([[-0.00137174,  0.01714678, -0.00205761, -0.00137174, -0.00137174,
         -0.00205761, -0.00205761, -0.00137174, -0.00137174, -0.00205761,
         -0.00205761, -0.00205761, -0.00205761,  0.01714678, -0.00205761,
         -0.00205761, -0.00137174, -0.00137174, -0.00137174, -0.00137174,
         -0.00205761, -0.00068587, -0.00068587],
        [ 0.01714678, -0.00137174,  0.01646091, -0.00137174, -0.00137174,
         -0.00205761, -0.00205761, -0.00137174, -0.00137174, -0.00205761,
         -0.00205761, -0.00205761, -0.00205761, -0.00137174, -0.00205761,
         -0.00205761, -0.00137174, -0.00137174, -0.00137174, -0.00137174,
         -0.00205761, -0.00068587, -0.00068587],
        [-0.00205761,  0.01646091, -0.00308642,  0.01646091, -0.00205761,
         -0.00308642, -0.00308642, -0.00205761, -0.00205761, -0.00308642,
         -0.00308642,  0.0154321 , -0.00308642, -0.00205761, -0.00308642,
         -0.00308642, -0.00205761, -0.00205761, -0.00205761, -0.00205761,
         -0.00

In [10]:
#covarianza para v 3.3
def attr_cov(v, cova, lag):
    cov = []
    for t in range(lag+1):
        cov.append((v.transpose() * cova[t] * v).tolist()[0][0])
    return cov

In [11]:
#covarianzar de variable categorica 3.6
def covarianza(Me, p, P, Ha, lag):
    # cov son las matrices de covarianza
    # aut son las matrices de autocovarianza
    # tr son las trazas de aut
    cov = []
    aut = []
    tr = []
    for t in range(lag +1):
        cov.append(ro(t, P, p, Me))
        aut.append(Ha.transpose() * cov[-1] * Ha)
        tr.append(np.trace(aut[-1]))
    return cov, aut, tr

In [12]:
evalues, evectors = sp_linalg.eigs(M.transpose(), k = 3, which = 'LM')
evectors

array([[ 0.17149859+0.j, -0.17149859+0.j, -0.09727702+0.j],
       [ 0.17149859+0.j,  0.17149859+0.j,  0.06678518+0.j],
       [ 0.25724788+0.j, -0.25724788+0.j, -0.04100714+0.j],
       [ 0.17149859+0.j,  0.17149859+0.j, -0.03107319+0.j],
       [ 0.17149859+0.j, -0.17149859+0.j,  0.08531786+0.j],
       [ 0.25724788+0.j,  0.25724788+0.j, -0.19218334+0.j],
       [ 0.25724788+0.j, -0.25724788+0.j,  0.36503288+0.j],
       [ 0.17149859+0.j,  0.17149859+0.j, -0.18153979+0.j],
       [ 0.17149859+0.j, -0.17149859+0.j,  0.09538161+0.j],
       [ 0.25724788+0.j,  0.25724788+0.j,  0.00534948+0.j],
       [ 0.25724788+0.j, -0.25724788+0.j,  0.04488549+0.j],
       [ 0.25724788+0.j,  0.25724788+0.j,  0.06120544+0.j],
       [ 0.25724788+0.j, -0.25724788+0.j, -0.1751841 +0.j],
       [ 0.17149859+0.j,  0.17149859+0.j,  0.11472496+0.j],
       [ 0.25724788+0.j,  0.25724788+0.j,  0.25702369+0.j],
       [ 0.25724788+0.j, -0.25724788+0.j, -0.20293038+0.j],
       [ 0.17149859+0.j,  0.17149859+0.j

In [13]:
#feature1: multi-hop assortativities of node IDs, setting H = I in Eq. (3.6) for 0, 1, ... , t hops
#H es la matriz identidad del tamaño del grafo
H = np.identity(len(d))
cov, aut, res1 = covarianza(M, pi, PI, H, l)
cov

[matrix([[ 0.03566529, -0.00137174, -0.00205761, -0.00137174, -0.00137174,
          -0.00205761, -0.00205761, -0.00137174, -0.00137174, -0.00205761,
          -0.00205761, -0.00205761, -0.00205761, -0.00137174, -0.00205761,
          -0.00205761, -0.00137174, -0.00137174, -0.00137174, -0.00137174,
          -0.00205761, -0.00068587, -0.00068587],
         [-0.00137174,  0.03566529, -0.00205761, -0.00137174, -0.00137174,
          -0.00205761, -0.00205761, -0.00137174, -0.00137174, -0.00205761,
          -0.00205761, -0.00205761, -0.00205761, -0.00137174, -0.00205761,
          -0.00205761, -0.00137174, -0.00137174, -0.00137174, -0.00137174,
          -0.00205761, -0.00068587, -0.00068587],
         [-0.00205761, -0.00205761,  0.05246914, -0.00205761, -0.00205761,
          -0.00308642, -0.00308642, -0.00205761, -0.00205761, -0.00308642,
          -0.00308642, -0.00308642, -0.00308642, -0.00205761, -0.00308642,
          -0.00308642, -0.00205761, -0.00205761, -0.00205761, -0.00205761,


In [14]:
#feature2: average of first p dominant left eigenvector of M, that is {πv : v is a dominant eigenvector of M}
print(len(cov))
res2 = attr_cov(pi.transpose(), cov, l)
res2

4


[0.00011101890896637645,
 9.408382115794642e-06,
 3.904478578054767e-05,
 -2.587305081843629e-06]

In [15]:
#feature3: multi-hop assortativities of the first p dominant left eigenvectors of M (Eq. 3.3) for 0, 1, ... , t hops
pi2 = np.asmatrix(evectors[:,1]).transpose()
h2 = pi2/np.linalg.norm(pi2, 1)
res3 = attr_cov(h2, cov, l)
pi3 = np.asmatrix(evectors[:,2]).transpose()
h3 = pi3/np.linalg.norm(pi3, 1)
res4 = attr_cov(h3, cov, l)

In [16]:
def avg_feats(Ha, p):
    avg = []
    for i in range(len(Ha[0])):
        avg.append(p * np.asmatrix(Ha[:,i]).transpose())
    return avg

In [17]:

if node_labels:
    #print(labels)
    unique, categorias = process_labels(labels)
    #print(categorias)
    cov, aut, tr = covarianza(M, pi, PI, categorias, l)
    #feature4: (If available) average of categorical metadata node attributes: {πhi : H = [h1, h2, ..hk ], 1 ≤ i ≤ k}
    res5 = np.array([np.diag(i) for i in aut]).flatten()
    #feature5: (If available) multi-hop assortativities of categorical metadata node attributes (Eq. 3.4) for 0, 1, ... , t hops
    res6 = avg_feats(categorias, pi)
    

In [18]:
#feature6: number of nodes
res9 = G.number_of_nodes()
#feature7: number of edges
res7 = G.number_of_edges()

r = {
    0: res1,
    1: [i.real for i in res2],
    2: [i.real for i in res3],
    3: [i.real for i in res4]
}
if node_labels:
    r[4] = res5
    r[5] = [el.tolist()[0][0] for el in res6]
r[6] = res7
r[7] = (pi * pi.transpose()).tolist()[0][0]
r[8] = res9
r[9] = (pi * pi2).tolist()[0][0].real
r[10] = (pi * pi3).tolist()[0][0].real
    
r

{0: [0.9533607681755829,
  -0.04663923182441701,
  0.3792866941015089,
  -0.04663923182441701],
 1: [0.00011101890896637645,
  9.408382115794642e-06,
  3.904478578054767e-05,
  -2.587305081843629e-06],
 2: [0.0022857664350323004,
  -0.0021850967463932966,
  0.0022137923118464705,
  -0.0021731010591956586],
 3: [0.0030827410537223312,
  -0.0025051608539724507,
  0.0025625901823950912,
  -0.0021893310966389153],
 4: array([ 0.0840192 ,  0.05246914,  0.03566529,  0.06550069, -0.00308642,
        -0.00137174,  0.05932785,  0.04012346,  0.02331962,  0.04903978,
        -0.00308642, -0.00137174]),
 5: [0.9074074074074074, 0.05555555555555555, 0.037037037037037035],
 6: 27,
 7: 0.04663923182441701,
 8: 23,
 9: 0.003175899724861329,
 10: -0.015428573669564918}

In [19]:
#create features dataframe
data = {}
titles = ['ID', 'pagerank', 'eigenvec_2nd_M', 'eigenvec_3rd_M', 'node_attribs', 'avg_node_labels', 'n_edges', 'avg_pi', 'avg_inv_pi', 'avg_eigenvec_2nd', 'avg_eigenvec_3rd']
for i in range(len(titles)):
    if type(r[i]) != float and type(r[i]) != int :
        for j in range(len(r[i])):
        
            data[titles[i] + '_' + str(j)] = [r[i][j]]
    else:
        data[titles[i]] = [r[i]]
df = pd.DataFrame(data)
df

,ID_0,ID_1,ID_2,ID_3,pagerank_0,pagerank_1,pagerank_2,pagerank_3,eigenvec_2nd_M_0,eigenvec_2nd_M_1,...,node_attribs_10,node_attribs_11,avg_node_labels_0,avg_node_labels_1,avg_node_labels_2,n_edges,avg_pi,avg_inv_pi,avg_eigenvec_2nd,avg_eigenvec_3rd
0,0.953361,-0.046639,0.379287,-0.046639,0.000111,0.000009,0.000039,-0.000003,0.002286,-0.002185,...,-0.003086,-0.001372,0.907407,0.055556,0.037037,27,0.046639,23,0.003176,-0.015429
